*Copyright: 
Prof. Dr. Christian Wartena, http://textmining.wp.hs-hannover.de/; Frieda Josi*

## Termextraktion
Die Frequenz der Terme mit der Wortart NN (Nomen) und VB (Verb in seiner Stammform) aus den Beschriftungen und Kontexttexte der 397 Abbildungen aus Artikel, die aus Open Access Journals (NOA Projekt http://noa.wp.hs-hannover.de) stammen. Die Artikel haben meist mehrere Abbildungen, die aber nicht alle mit aufgenommen wurden. Es sind 397 einzelne txt-Dateien, angezeigt werden die Datensätze 150 bis 170. 

### Vorkommenshäufigkeit der Terme

In [1]:
import glob
import nltk
import codecs
import pprint
import treetaggerwrapper


def extract_kw(datei):
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
    
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()

    sentences = nltk.sent_tokenize(text)#Aufteilen des Datensatzes in Sätze
    sentences_tok = [nltk.word_tokenize(sent) for sent in sentences] #Aufteilen der Sätze in Wörter
    nouns = [] 
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) #Erstellt Liste mit: Token+Tag z.B. tag\tVV\ttag 
        tags2 = treetaggerwrapper.make_tags(tags); #Erstellt Tupel daraus z.B. Tag(word='tag', pos='VB', lemma='tag')
        nouns_from_sent = [lemma for (word,pos,lemma) in tags2 if pos == "NN" or pos == "NNS" or pos == "VB"] # Grundform von Nomen, Verb in Grundform
        nouns.extend(nouns_from_sent) #Terme werden in Liste nouns hinzugefügt
               
    fdist = nltk.FreqDist(nouns)#Anzahl Vorkommen der Terme (bestehend aus Nomen oder Verben)
    
    return fdist.most_common(15)#Anzahl der Ausgabe der Keywords pro Abbildung (Beschriftung inkl. Textreferenzen)
 

filelist = glob.glob("doi-cap-con/*.txt")#glob-Modul für Pfadnamen nach Muster
for f in filelist[150:170]: #Anzahl der Dateien 396, aufgerufen werden hier 20
    terme = extract_kw(f) #Funktion extract_term(), aufgerufen mit jeder Datei (f)
    pprint.pprint(f) #Ausgabe Name der Textdatei
    pprint.pprint(terme) #Ausgabe der Terme + Anzahl Vorkommen der Wörter
    pprint.pprint("------------------------------")
    
    
    

'doi-cap-con/10.1155:2016:1875357.txt'
[('stripe', 7),
 ('slot', 6),
 ('size', 6),
 ('wavelength', 5),
 ('datum', 4),
 ('time', 4),
 ('system', 3),
 ('fabric', 3),
 ('switch', 3),
 ('bit', 3),
 ('computer', 2),
 ('consideration', 2),
 ('control', 2),
 ('switching', 2),
 ('be', 2)]
'------------------------------'
'doi-cap-con/10.1155:2016:1901493.txt'
[('ball', 4),
 ('condition', 3),
 ('tribotester', 3),
 ('four-ball', 2),
 ('lubricant', 2),
 ('contact', 2),
 ('kg', 2),
 ('lubrication', 1),
 ('place', 1),
 ('load', 1),
 ('spindle', 1),
 ('engine', 1),
 ('oil', 1),
 ('test', 1),
 ('geometry', 1)]
'------------------------------'
'doi-cap-con/10.1155:2016:1925827.txt'
[('rectangle', 5),
 ('distribution', 5),
 ('space', 4),
 ('condition', 4),
 ('k', 4),
 ('state', 4),
 ('lattice', 3),
 ('=P', 3),
 ('range', 3),
 ('boundary', 3),
 ('m', 2),
 ('transition', 2),
 ('ρ→∞', 2),
 ('walk', 2),
 ('be', 2)]
'------------------------------'
'doi-cap-con/10.1155:2016:1928465.txt'
[('curve', 2),
 ('we

## Termfrequenz
Termfrequenz statt einfache Vorkommenshäufigkeit der Terme. Anzahl Termvorkommen geteilt durch die Länge des Textes (Datensatzes).  

In [2]:
import glob
import nltk
import codecs
import pprint
import treetaggerwrapper
import math

df = {} 
tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')

def substantive_zaehlen(datei): #Anzahl der Substantive in einer Datei zählen, 397 Dokumente
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    nouns_in_text = [] #Liste für Häufigkeit/Anzahl von allen NN+VB in einem Text
    
    sentences = nltk.sent_tokenize(text) #Aufteilen des Textes in Sätze
    sentences_tok = [nltk.word_tokenize(sent) for sent in sentences] #Aufteilen der Sätze in Wörter
    nouns = [] 
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) #Erstellt Liste mit: Token+Tag z.B. tag\tVV\ttag 
        tags2 = treetaggerwrapper.make_tags(tags); #Erstellt Tupel daraus z.B. Tag(word='tag', pos='VB', lemma='tag')
        nouns_from_sent = [lemma for (word,pos,lemma) in tags2 if pos == "NN" or pos == "NNS" or pos == "VB"] #Stammform von NN und VB
        for substantiv in nouns_from_sent:
            if substantiv not in nouns_in_text: 
                nouns_in_text.append(substantiv) #Stammform der NN + VB werden der Liste nouns_in_text einmal hinzugefügt
                
    for n in nouns_in_text: #Für jedes Substantiv im Text die Häufigkeit für n
        df_n = df.get(n,0) #Erstes Argument ist der Schlüssel, das zweite Argument ist der Wert, der zurückgegeben wird, falls der Schlüssel im Dictionary nicht vorhanden ist 
        df[n] = df_n + 1 #Neuer Wert (+Schlüssel) wird dem Dictionary hinzugefügt
    

for f in glob.glob("doi-cap-con/*.txt"): #Anwenden auf alle Texte
    substantive_zaehlen(f) #Funktion substantive_zaehlen(), aufgerufen mit jeder Datei (f)
    

def extract_term(datei,nr_of_docs): #Funktion für Berechnung der inversen Dokumentfrequenz 
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
    
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    sentences = nltk.sent_tokenize(text)#Aufteilen der Datensätze in Sätze
    sentences_tok = [nltk.word_tokenize(sent) for sent in sentences]#Aufteilen der Sätze in Wörter
    nouns = [] #Liste
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) #Erstellt Liste mit: Token+Tag z.B. tag\tVV\ttag 
        tags2 = treetaggerwrapper.make_tags(tags); #Erstellt Tupel daraus z.B. Tag(word='tag', pos='VB', lemma='tag')
        nouns_from_sent = [lemma for (word,pos,lemma) in tags2 if pos == "NN" or pos == "NNS" or pos == "VB"] #Grundform von NN, NNS und VB
        nouns.extend(nouns_from_sent)#Terme werden in Liste nouns hinzugefügt
    fdist = nltk.FreqDist(nouns)  #Anzahl Vorkommen der Terme
    
    for word in fdist: #Normalisierte Termfrequenz für Grundformen von NN+VB+NNS
            fdist[word] = float(fdist[word]) / float(len(nouns)) #Anzahl der Wörter durch Länge aller Wörter in einem Datensatz
        
    
    return fdist.most_common(15) #Die häufigsten 15
 
filelist = glob.glob("doi-cap-con/*.txt") #glob-Modul für Pfadnamen nach Muster
nr_of_docs = len(filelist) 
for f in filelist[150:170]: #Anzahl der Dateien 397, aufgerufen werden hier 20 
    terme = extract_term(f,nr_of_docs) #Funktion extract_term(), aufgerufen mit jeder Datei (f)
    pprint.pprint(f) #Ausgabe Name der Textdatei
    pprint.pprint(terme) #Ausgabe der Terme + Anzahl Vorkommen der Wörter (tf-idf)
    pprint.pprint("------------------------------")
    
    


'doi-cap-con/10.1155:2016:1875357.txt'
[('stripe', 0.07954545454545454),
 ('slot', 0.06818181818181818),
 ('size', 0.06818181818181818),
 ('wavelength', 0.056818181818181816),
 ('datum', 0.045454545454545456),
 ('time', 0.045454545454545456),
 ('system', 0.03409090909090909),
 ('fabric', 0.03409090909090909),
 ('switch', 0.03409090909090909),
 ('bit', 0.03409090909090909),
 ('computer', 0.022727272727272728),
 ('consideration', 0.022727272727272728),
 ('control', 0.022727272727272728),
 ('switching', 0.022727272727272728),
 ('be', 0.022727272727272728)]
'------------------------------'
'doi-cap-con/10.1155:2016:1901493.txt'
[('ball', 0.08888888888888889),
 ('condition', 0.06666666666666667),
 ('tribotester', 0.06666666666666667),
 ('four-ball', 0.044444444444444446),
 ('lubricant', 0.044444444444444446),
 ('contact', 0.044444444444444446),
 ('kg', 0.044444444444444446),
 ('lubrication', 0.022222222222222223),
 ('place', 0.022222222222222223),
 ('load', 0.022222222222222223),
 ('spindle

### Inverse Dokumenthäufigkeit:  
(en: inverse Document Frequency (IDF))
Tf-idf-Werte Berechnung für Substantive+Verben
Terme der Datensätze: Wortformen NN, NNS und VB in ihrer Grundform

In [3]:
import glob
import nltk
import codecs
import pprint
import treetaggerwrapper
import math

df = {} 
tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')

def substantive_zaehlen(datei): #Anzahl der Substantive in einer Datei zählen, 397 Dokumente
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    nouns_in_text = [] #Liste für Häufigkeit/Anzahl von allen NN+NNS+VB in einem Text
    
    sentences = nltk.sent_tokenize(text) #Aufteilen des Textes in Sätze
    sentences_tok = [nltk.word_tokenize(sent) for sent in sentences] #Aufteilen der Sätze in Wörter
    nouns = [] 
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) #Erstellt Liste mit: Token+Tag z.B. tag\tVV\ttag 
        tags2 = treetaggerwrapper.make_tags(tags); #Erstellt Tupel daraus z.B. Tag(word='tag', pos='VB', lemma='tag')
        nouns_from_sent = [lemma for (word,pos,lemma) in tags2 if pos == "NN" or pos == "NNS" or pos == "VB"] #Stammform von NN, NNS und VB
        for substantiv in nouns_from_sent:
            if substantiv not in nouns_in_text: 
                nouns_in_text.append(substantiv) #Stammform der NN, NNS + VB werden der Liste nouns_in_text einmal hinzugefügt
                
    for n in nouns_in_text: #Für jedes Substantiv im Text die Häufigkeit für n 
        df_n = df.get(n,0) #Erstes Argument ist der Schlüssel, das zweite Argument ist der Wert, der zurückgegeben wird, falls der Schlüssel im Dictionary nicht vorhanden ist 
        df[n] = df_n + 1 #Neuer Wert (+Schlüssel) wird dem Dictionary hinzugefügt
    

for f in glob.glob("doi-cap-con/*.txt"): #Anwenden auf alle Texte
    substantive_zaehlen(f) #Funktion substantive_zaehlen(), aufgerufen mit jeder Datei (f)
    

def extract_term(datei,nr_of_docs): #Funktion für Berechnung der inversen Dokumentfrequenz (idf)
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
    
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    sentences = nltk.sent_tokenize(text)#Aufteilen der Datensätze in Sätze
    sentences_tok = [nltk.word_tokenize(sent) for sent in sentences]#Aufteilen der Sätze in Wörter
    nouns = [] 
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) #Erstellt Liste mit: Token+Tag z.B. tag\tVV\ttag 
        tags2 = treetaggerwrapper.make_tags(tags); #Erstellt Tupel daraus z.B. Tag(word='tag', pos='VB', lemma='tag')
        nouns_from_sent = [lemma for (word,pos,lemma) in tags2 if pos == "NN" or pos == "NNS" or pos == "VB"] #Grundform von NN, NNS und VB 
        nouns.extend(nouns_from_sent)#Terme werden in Liste nouns hinzugefügt
    fdist = nltk.FreqDist(nouns)  #Anzahl Vorkommen der Terme
    
    for word in fdist: #Tf-idf-Maß Berechnung für alle Grundformen von NN+NNS+VB
        idf = 1.0 + math.log(float(nr_of_docs) / float(df[word])) #Log von (Anzahl Dokumente dividiert durch Wort+Schlüssel)
        fdist[word] = float(fdist[word]) / float(len(nouns)) * idf #Anzahl der Wörter durch Länge aller Wörter mulipliziert mit idf (s.o.) 
 
    
    return fdist.most_common(15) #Die häufigsten 15
 
filelist = glob.glob("doi-cap-con/*.txt") #glob-Modul für Pfadnamen nach Muster
nr_of_docs = len(filelist) #Länge der Dateiliste
for f in filelist[150:170]: #Anzahl der Dateien 397, aufgerufen werden hier 20 
    terme = extract_term(f,nr_of_docs) #Funktion extract_term() aufgerufen mit jeder Datei (f)
    pprint.pprint(f) #Ausgabe Name der Textdatei
    pprint.pprint(terme) #Ausgabe der Terme + Anzahl Vorkommen der Wörter (tf-idf)
    pprint.pprint("------------------------------")
    

'doi-cap-con/10.1155:2016:1875357.txt'
[('stripe', 0.5553397668043337),
 ('slot', 0.4760055144037145),
 ('wavelength', 0.3179045369394653),
 ('fabric', 0.23800275720185726),
 ('bit', 0.23800275720185726),
 ('size', 0.2092766731245228),
 ('switch', 0.17692004802362812),
 ('switching', 0.14291515978851216),
 ('computer', 0.12716181477578614),
 ('conversion', 0.12716181477578614),
 ('time', 0.11330808370556096),
 ('advantage', 0.11140846976306013),
 ('datum', 0.10579335732258896),
 ('system', 0.1026518965125804),
 ('consideration', 0.10219335366969272)]
'------------------------------'
'doi-cap-con/10.1155:2016:1901493.txt'
[('ball', 0.5229157264521218),
 ('tribotester', 0.46542761408363204),
 ('four-ball', 0.3102850760557547),
 ('lubricant', 0.3102850760557547),
 ('kg', 0.23065132186784112),
 ('condition', 0.22119017100832228),
 ('contact', 0.20794796081157488),
 ('lubrication', 0.15514253802787736),
 ('combustion', 0.15514253802787736),
 ('follower', 0.15514253802787736),
 ('three-point

# Mapping mit Wikipedia Artikel Titel
Tf-idf Berechnung der Terme. Abgleich der Terme mit Wikipedia Artikel-Titeln, Rückgabe der Kategorien der Artikel

In [4]:
import glob
import nltk
import codecs
import pprint
import treetaggerwrapper
import math
import requests
from nltk.corpus import stopwords 

filelist = glob.glob("doi-cap-con/*.txt") #glob-Modul für Pfadnamen nach Muster
nr_of_docs = len(filelist) #Länge der Dateiliste
noun2cat = {} #Dictionary für Substantiv -> [Wikicats]
catStoplist = stopwords.words('categories') #Kategorien-Stoppliste (z.B Numbers)
df = {} 
tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')

def substantive_zaehlen(datei): #Anzahl der Substantive in einer Datei zählen, 397 Dokumente
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    nouns_in_text = [] #Liste für Häufigkeit/Anzahl von allen NN+NNS+VB in einem Text
    
    sentences = nltk.sent_tokenize(text) #Aufteilen des Textes in Sätze
    sentences_tok = [nltk.word_tokenize(sent) for sent in sentences] #Aufteilen der Sätze in Wörter
    nouns = [] 
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) #Erstellt Liste mit: Token+Tag z.B. tag\tVV\ttag 
        tags2 = treetaggerwrapper.make_tags(tags); #Erstellt Tupel daraus z.B. Tag(word='tag', pos='VB', lemma='tag')
        nouns_from_sent = [lemma for (word,pos,lemma) in tags2 if pos == "NN" or pos == "NNS" or pos == "VB"] #Stammform von NN, NNS und VB
        for substantiv in nouns_from_sent:
            if substantiv not in nouns_in_text: 
                nouns_in_text.append(substantiv) #Stammform der NN + VB werden der Liste nouns_in_text einmal hinzugefügt
                
    for n in nouns_in_text: #Für jedes Substantiv im Text hole die Häufigkeit für n raus
        df_n = df.get(n,0) #Erstes Argument ist der Schlüssel, das zweite Argument ist der Wert, der zurückgegeben wird, falls der Schlüssel im Dictionary nicht vorhanden ist 
        df[n] = df_n + 1 #Neuer Wert (+Schlüssel) wird dem Dictionary hinzugefügt

for f in glob.glob("doi-cap-con/*.txt"): #Anwenden auf alle Texte
    substantive_zaehlen(f) #Funktion substantive_zaehlen oben, aufgerufen mit jeder Datei (f)

    
def extract_term(datei,nr_of_docs): #Funktion für Berechnung der inversen Dokumentfrequenz 
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
    
    textfile = codecs.open(datei, "r", "utf8")
    text = textfile.read()
    textfile.close()
    
    sentences = nltk.sent_tokenize(text)#Aufteilen des Textes in Sätze
    sentences_tok = [nltk.word_tokenize(sent) for sent in sentences]#Aufteilen der Sätze in Wörter
    nouns = [] 
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) #Erstellt Liste mit: Token+Tag z.B. tag\tVV\ttag 
        tags2 = treetaggerwrapper.make_tags(tags); #Erstellt Tupel daraus z.B. Tag(word='tag', pos='VB', lemma='tag')
        nouns_from_sent = [lemma for (word,pos,lemma) in tags2 if pos == "NN" or pos == "NNS" or pos == "VB"] #Grundform von NN und VB
        nouns.extend(nouns_from_sent)#Terme werden in Liste nouns hinzugefügt
    fdist = nltk.FreqDist(nouns)  #Anzahl Vorkommen der Terme
    
    for word in fdist: #Tf-idf-Maß Berechnung für alle Grundformen von NN+NNS+VB
        idf = 1.0 + math.log(float(nr_of_docs) / float(df[word])) #Log von (Anzahl Dokumente dividiert durch Wort+Schlüssel)
        fdist[word] = float(fdist[word]) / float(len(nouns)) * idf #Anzahl der Wörter durch Länge aller Wörter mulipliziert mit idf (s.o.)        
    
    return fdist.most_common(15) #Die besten 5/10/15 Terme

def wiki_cats(term):
    cats = []
    
    response = requests.get(
        'https://en.wikipedia.org/w/api.php',
        params={
            'action': 'query',
            'format': 'json',
            'titles': term,
            'prop': 'categories',
            'clshow':'!hidden',
            'cllimit':'50'
        }
    ).json() 
    
    for pageid in  response['query']['pages']:
        if len(term) >2 :   #Terme sollen eine Länge von über 2 Buchstaben haben 
            if 'categories' in response['query']['pages'][pageid]:
                cats.extend([cat['title'] for cat in response['query']['pages'][pageid]['categories'] if cat['ns'] == 14 and not cat['title'].startswith('Category:Disambiguation') and cat['title'] not in catStoplist])

    return set(cats) #Mit set werden Duplikate entfernt

def collect_nouns(flist):
    global noun2cat
    for f in flist:
        nouns = extract_term(f, nr_of_docs)
        for (n,f) in nouns:
            noun2cat[n] = wiki_cats(n)
 

collect_nouns(filelist[150:170])


for f in filelist[150:170]: #Anzahl der Dateien 397, aufgerufen werden hier 20 
    terme = extract_term(f,nr_of_docs) #Funktion extract_term oben, aufgerufen mit jeder Datei (f)
    cats = set([c for (kw,f) in terme for c in noun2cat.get(kw)]) 
    pprint.pprint(f) 
    pprint.pprint(cats)
    pprint.pprint("------------------------------")

'doi-cap-con/10.1155:2016:1875357.txt'
{'Category:1990s fads and trends',
 'Category:2000s fads and trends',
 'Category:Binary arithmetic',
 'Category:Computers',
 'Category:Concepts in metaphysics',
 'Category:Concepts in physics',
 'Category:Consumer electronics',
 'Category:Contract law',
 'Category:Data types',
 'Category:Electrical components',
 'Category:Human–machine interaction',
 'Category:Legal doctrines and principles',
 'Category:Physical quantities',
 'Category:Primitive types',
 'Category:SI base quantities',
 'Category:Size',
 'Category:Switches',
 'Category:Systems',
 'Category:Units of information',
 'Category:Waves'}
'------------------------------'
'doi-cap-con/10.1155:2016:1901493.txt'
{'Category:Balls',
 'Category:Classical mechanics',
 'Category:Combustion',
 'Category:Force',
 'Category:Friction',
 'Category:Lubricants',
 'Category:Lubrication',
 'Category:Mechanical engineering',
 'Category:Petroleum products',
 'Category:Tribology'}
'---------------------------

### Ranking der Kategorien
Oberkategorien der Kategorien ermitteln. Anzahl der Überscheidungen = Anzahl Beziehungen
Ranking nach Anzahl

In [5]:
def intersection(a,b):
    i = 0
    for e in a:
        if e in b:
            i += 1
    return i

def wiki_cats_sort_rel(f):
    terme = extract_term(f,nr_of_docs) 
    cats = set([c for (kw,f) in terme for c in noun2cat.get(kw)]) #Mit set werden doppelte Kategorien entfernt, Menge
    extended = {}
    relnumber = {}
    for cat in cats:
        family = [cat]
        family.extend(wiki_cats(cat)) #Funktion wiki_cats(), alle Kategorien eines Artikels ohne Stoppliste,...
        extended[cat] = family

    for cat in cats:
        relsize = 0
        for cat2 in cats:
            if cat == cat2:
                continue
            if intersection(extended[cat],extended[cat2]) > 0:
                relsize +=1
        relnumber[cat] = relsize
    return sorted(relnumber.items(),key = lambda x:x[1],reverse=True)
            
for f in filelist[150:170]:    
    pprint.pprint(f) 
    cts = wiki_cats_sort_rel(f) 
    pprint.pprint(cts)#Ausgabe der Kategorien, sortiert nit Anzahl der Überscheidungen
    pprint.pprint("------------------------------") 



'doi-cap-con/10.1155:2016:1875357.txt'
[('Category:1990s fads and trends', 1),
 ('Category:Systems', 1),
 ('Category:2000s fads and trends', 1),
 ('Category:Switches', 1),
 ('Category:Concepts in metaphysics', 1),
 ('Category:Electrical components', 1),
 ('Category:Concepts in physics', 1),
 ('Category:Data types', 1),
 ('Category:Physical quantities', 1),
 ('Category:Primitive types', 1),
 ('Category:Units of information', 0),
 ('Category:Human–machine interaction', 0),
 ('Category:Waves', 0),
 ('Category:Computers', 0),
 ('Category:Binary arithmetic', 0),
 ('Category:Consumer electronics', 0),
 ('Category:SI base quantities', 0),
 ('Category:Size', 0),
 ('Category:Legal doctrines and principles', 0),
 ('Category:Contract law', 0)]
'------------------------------'
'doi-cap-con/10.1155:2016:1901493.txt'
[('Category:Tribology', 2),
 ('Category:Force', 2),
 ('Category:Friction', 2),
 ('Category:Mechanical engineering', 1),
 ('Category:Lubricants', 1),
 ('Category:Lubrication', 1),
 ('Cat